In [1]:
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import create_engine

In [2]:
file = "EPA_Info.csv"

In [3]:
epa_data_df = pd.read_csv(file)
epa_data_df.head()

,Date,Carbon Monoxide AQI Value,Main Pollutant,Site Name,Site ID,Source,Nitrogen Dioxide AQI Value,Main Pollutant.1,Site Name.1,Site ID.1,...,Sulfur Dioxide AQI Value,Main Pollutant.4,Site Name.4,Site ID.4,Source.4,PM10 AQI Value,Main Pollutant.5,Site Name.5,Site ID.5,Source.5
0,1/1/2017,11,NaN,Elizabeth Lab,34-039-0004,AQS,44,NaN,Elizabeth Lab,34-039-0004,...,4,NaN,PFIZER LAB SITE,36-005-0133,AQS,NaN,NaN,NaN,NaN,NaN
1,1/2/2017,13,NaN,Elizabeth,34-039-0003,AQS,43,NaN,Bayonne,34-017-0006,...,17,NaN,EISENHOWER PARK,36-059-0005,AQS,NaN,NaN,NaN,NaN,NaN
2,1/3/2017,7,NaN,Elizabeth,34-039-0003,AQS,42,NaN,Elizabeth Lab,34-039-0004,...,1,NaN,PFIZER LAB SITE,36-005-0133,AQS,NaN,NaN,NaN,NaN,NaN
3,1/4/2017,6,NaN,Jersey City,34-017-1002,AQS,27,NaN,Fort Lee Near Road,34-003-0010,...,3,NaN,Chester,34-027-3001,AQS,7.0,NaN,Newark Firehouse,34-013-0003,AQS
4,1/5/2017,6,NaN,Jersey City,34-017-1002,AQS,28,NaN,Jersey City,34-017-1002,...,3,NaN,QUEENS COLLEGE 2,36-081-0124,AQS,NaN,NaN,NaN,NaN,NaN


In [4]:
epa_data_df['Year'] = pd.DatetimeIndex(epa_data_df['Date']).year


In [5]:
new_epadata_df = epa_data_df[['Year', 'Sulfur Dioxide AQI Value',  'Nitrogen Dioxide AQI Value','Ozone AQI Value','Carbon Monoxide AQI Value','PM10 AQI Value', 'PM2.5 AQI Value']].copy()

city = ['New York', 'New York', 'New York'] 
clean_df = new_epadata_df.groupby('Year', as_index=False).mean()
clean_df['City'] = city
clean_df = clean_df.rename(columns={'Sulfur Dioxide AQI Value': 'SO2','Nitrogen Dioxide AQI Value':'NO2', 'Ozone AQI Value':'O3', 'Carbon Monoxide AQI Value': 'CO','PM10 AQI Value': 'PM10', 'PM2.5 AQI Value': 'PM2.5'})

clean_df.style.format({'SO2': "{:,.2f}", 'NO2': '{:.2f}', 'O3': '{:.2f}', 'CO': '{:,.2f}', 'PM10': '{:,.2f}', 'PM2.5': '{:,.2f}'})



,Year,SO2,NO2,O3,CO,PM10,PM2.5,City
0,2017,2.88,38.13,46.11,7.21,14.60,49.54,New York
1,2018,3.05,38.39,47.12,8.16,14.35,50.06,New York
2,2019,3.04,38.81,46.92,7.85,14.13,47.67,New York


In [6]:
rds_connection_string = "postgres:postgres@localhost:5432/NYEPA_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [7]:
engine.table_names()

[]

In [8]:
clean_df.to_sql(name='NYPollution', con=engine, if_exists='append', index=False)
